<a href="https://colab.research.google.com/github/MUTHYALA-SANJAYREDDY/IMAGE_STEGANOGRAPHY/blob/main/IMAGE_STEGANOGRAPHY.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import hashlib

def hash_file_md5(input_file_path, output_file_path):
    try:
        # Read the content of the input file
        with open(input_file_path, 'r') as file:
            file_content = file.read()

        # Perform MD5 hashing
        md5_hash = hashlib.md5(file_content.encode()).hexdigest()

        # Save the MD5 hash in the output file
        with open(output_file_path, 'w') as output_file:
            output_file.write(md5_hash)

        print(f"MD5 hash of the file saved to {output_file_path}")

    except FileNotFoundError:
        print("The specified input file does not exist.")
    except Exception as e:
        print(f"An error occurred: {e}")

# Example usage
input_file_path = '/content/sample_data/input.txt'
output_file_path = '/content/sample_data/md5_hash.txt'
hash_file_md5(input_file_path, output_file_path)


MD5 hash of the file saved to /content/sample_data/md5_hash.txt


In [ ]:
import hashlib

def double_hash_md5_sha256(input_file_path, output_file_path):
    try:
        # Read the MD5 hash from the input file
        with open(input_file_path, 'r') as file:
            md5_hash = file.read().strip()  # Ensure no extra whitespace

        # Perform SHA-256 hashing on the MD5 hash
        sha256_hash = hashlib.sha256(md5_hash.encode()).hexdigest()

        # Save the double-hashed value (SHA-256 of MD5) to the output file
        with open(output_file_path, 'w') as output_file:
            output_file.write(sha256_hash)

        print(f"Double-hashed value (SHA-256 of MD5) saved to {output_file_path}")

    except FileNotFoundError:
        print("The specified input file does not exist.")
    except Exception as e:
        print(f"An error occurred: {e}")

# Example usage
input_file_path = '/content/sample_data/md5_hash.txt'
output_file_path = '/content/sample_data/double_hash.txt'
double_hash_md5_sha256(input_file_path, output_file_path)


Double-hashed value (SHA-256 of MD5) saved to /content/sample_data/double_hash.txt


In [ ]:
!pip install cryptography
!pip install pycryptodome

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 21.9 MB/s eta 0:00:00


In [ ]:
from Crypto.Random import get_random_bytes

def generate_aes_key_and_save(output_file):
    # Step 1: Generate a random AES-128 key (16 bytes)
    aes_key = get_random_bytes(16)  # AES-128 uses 16 bytes

    # Step 2: Save the raw key directly into the file (binary format, or hex for readability)
    with open(output_file, 'wb') as f:  # Save in binary
        f.write(aes_key)

    # Also print the AES key as a hex string for verification
    aes_key_hex = aes_key.hex()
    print(f"AES-128 Key (16 bytes, Hex): {aes_key_hex}")

# Example usage
output_file = "/content/sample_data/aes_key.bin"  # Path to save the generated AES-128 key

generate_aes_key_and_save(output_file)


AES-128 Key (16 bytes, Hex): cf73cbe1ef5b8d24db6ab7fe616ea08e


In [ ]:
import os
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes
from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives import padding

def read_aes_key(key_file_path):
    try:
        # Read the AES key from the provided file
        with open(key_file_path, 'rb') as key_file:
            aes_key = key_file.read()

        if len(aes_key) != 16:
            raise ValueError("Invalid AES key length. The key must be 16 bytes (128-bit) for AES-128 encryption.")

        print(f"Read AES-128 Key (hex): {aes_key.hex()}")
        return aes_key
    except FileNotFoundError:
        print(f"The key file at {key_file_path} does not exist.")
        raise
    except Exception as e:
        print(f"An error occurred while reading the AES key: {e}")
        raise

def aes_encrypt(plaintext, key):
    # Generate a random initialization vector (IV) for AES
    iv = os.urandom(16)
    print(f"Generated IV (hex): {iv.hex()}")

    # Create the AES cipher in CBC mode
    cipher = Cipher(algorithms.AES(key), modes.CBC(iv), backend=default_backend())
    encryptor = cipher.encryptor()

    # Pad the plaintext to be a multiple of AES block size (128-bit)
    padder = padding.PKCS7(128).padder()
    padded_plaintext = padder.update(plaintext) + padder.finalize()
    print(f"Padded Plaintext (hex): {padded_plaintext.hex()}")

    # Encrypt the padded plaintext
    ciphertext = encryptor.update(padded_plaintext) + encryptor.finalize()
    print(f"Ciphertext (hex): {ciphertext.hex()}")

    # Return the IV and ciphertext separately
    return iv, ciphertext

def encrypt_file(input_file_path, key_file_path, output_file_path, iv_file_path):
    try:
        # Read the content of the input file
        with open(input_file_path, 'rb') as file:
            plaintext = file.read()
        print(f"Original Plaintext: {plaintext.decode('utf-8')}")
        print(f"Original Plaintext (hex): {plaintext.hex()}")

        # Read the AES-128 key from the key file
        aes_key = read_aes_key(key_file_path)

        # Encrypt the file's content using the provided AES-128 key
        iv, ciphertext = aes_encrypt(plaintext, aes_key)

        # Save the encrypted ciphertext to the output file (in hex format)
        with open(output_file_path, 'w') as output_file:
            output_file.write(ciphertext.hex())  # Save ciphertext as hex
        print(f"\nCiphertext saved (hex) to {output_file_path}")

        # Save the generated IV to a separate file (in hex format)
        with open(iv_file_path, 'w') as iv_file:
            iv_file.write(iv.hex())  # Save IV as hex
        print(f"Initialization Vector (IV) saved (hex) to {iv_file_path}")

    except FileNotFoundError:
        print("The specified input file does not exist.")
    except Exception as e:
        print(f"An error occurred: {e}")

# Example usage
input_file_path = '/content/sample_data/input.txt'
key_file_path = '/content/sample_data/aes_key.bin'
output_file_path = '/content/sample_data/aes128_ciphertext_file.txt'
iv_file_path = '/content/sample_data/aes128_iv_file.txt'
encrypt_file(input_file_path, key_file_path, output_file_path, iv_file_path)


Original Plaintext: hi, how are you?
Original Plaintext (hex): 68692c20686f772061726520796f753f
Read AES-128 Key (hex): cf73cbe1ef5b8d24db6ab7fe616ea08e
Generated IV (hex): bd4158673313d21106f12f976ea159c1
Padded Plaintext (hex): 68692c20686f772061726520796f753f10101010101010101010101010101010
Ciphertext (hex): 0323774c7cb83f4b837f41ed0b001d461f9207745269940c5afeb62616fadf6a

Ciphertext saved (hex) to /content/sample_data/aes128_ciphertext_file.txt
Initialization Vector (IV) saved (hex) to /content/sample_data/aes128_iv_file.txt


In [ ]:
import os
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes
from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives import padding

def read_aes_key(key_file_path):
    try:
        # Read the AES key from the provided file
        with open(key_file_path, 'rb') as key_file:
            aes_key = key_file.read()

        if len(aes_key) != 16:
            raise ValueError("Invalid AES key length. The key must be 16 bytes (128-bit) for AES-128 encryption.")

        print(f"Read AES-128 Key (hex): {aes_key.hex()}")
        return aes_key
    except FileNotFoundError:
        print(f"The key file at {key_file_path} does not exist.")
        raise
    except Exception as e:
        print(f"An error occurred while reading the AES key: {e}")
        raise

def aes_encrypt(plaintext, key):
    # Generate a random initialization vector (IV) for AES
    iv = os.urandom(16)
    print(f"Generated IV (hex): {iv.hex()}")

    # Create the AES cipher in CBC mode
    cipher = Cipher(algorithms.AES(key), modes.CBC(iv), backend=default_backend())
    encryptor = cipher.encryptor()

    # Pad the plaintext to be a multiple of AES block size (128-bit)
    padder = padding.PKCS7(128).padder()
    padded_plaintext = padder.update(plaintext) + padder.finalize()
    print(f"Padded Plaintext (hex): {padded_plaintext.hex()}")

    # Encrypt the padded plaintext
    ciphertext = encryptor.update(padded_plaintext) + encryptor.finalize()
    print(f"Ciphertext (hex): {ciphertext.hex()}")

    # Return the IV and ciphertext separately
    return iv, ciphertext

def encrypt_file(input_file_path, key_file_path, output_file_path, iv_file_path):
    try:
        # Read the content of the input file
        with open(input_file_path, 'rb') as file:
            plaintext = file.read()
        print(f"Original Plaintext: {plaintext.decode('utf-8')}")
        print(f"Original Plaintext (hex): {plaintext.hex()}")

        # Read the AES-128 key from the key file
        aes_key = read_aes_key(key_file_path)

        # Encrypt the file's content using the provided AES-128 key
        iv, ciphertext = aes_encrypt(plaintext, aes_key)

        # Save the encrypted ciphertext to the output file (in hex format)
        with open(output_file_path, 'w') as output_file:
            output_file.write(ciphertext.hex())  # Save ciphertext as hex
        print(f"\nCiphertext saved (hex) to {output_file_path}")

        # Save the generated IV to a separate file (in hex format)
        with open(iv_file_path, 'w') as iv_file:
            iv_file.write(iv.hex())  # Save IV as hex
        print(f"Initialization Vector (IV) saved (hex) to {iv_file_path}")

    except FileNotFoundError:
        print("The specified input file does not exist.")
    except Exception as e:
        print(f"An error occurred: {e}")

# Example usage
input_file_path = '/content/sample_data/input.txt'            # Path to your plaintext input file
key_file_path = '/content/sample_data/aes_key.bin'            # Path to your AES-128 key file
output_file_path = '/content/sample_data/aes128_ciphertext_file.txt'  # Path to save the ciphertext
iv_file_path = '/content/sample_data/aes128_iv_file.txt'             # Path to save the IV

encrypt_file(input_file_path, key_file_path, output_file_path, iv_file_path)


Original Plaintext: hi, how are you?
Original Plaintext (hex): 68692c20686f772061726520796f753f
Read AES-128 Key (hex): cf73cbe1ef5b8d24db6ab7fe616ea08e
Generated IV (hex): 2bc6ac78d515f4ec1d5ee192590a81d3
Padded Plaintext (hex): 68692c20686f772061726520796f753f10101010101010101010101010101010
Ciphertext (hex): cb626d3c3c72d85df15c443a2ac86f17e928df1fc7c834aec242f06339788f78

Ciphertext saved (hex) to /content/sample_data/aes128_ciphertext_file.txt
Initialization Vector (IV) saved (hex) to /content/sample_data/aes128_iv_file.txt


In [ ]:
def read_hex_file_as_bin(file_path):
    """Reads a file containing hex data and returns its content in binary format."""
    with open(file_path, 'r') as file:
        hex_data = file.read().strip()  # Strip any extra whitespace or newlines
        return bytes.fromhex(hex_data)  # Convert hex to binary

def read_file_as_bin(file_path):
    """Reads a binary file and returns its content as a binary string."""
    with open(file_path, 'rb') as file:
        return file.read()

def save_hex_to_file(hex_data, output_path):
    """Saves the hexadecimal data to a text file."""
    with open(output_path, 'w') as file:
        file.write(hex_data)

def concatenate_and_save_in_hex(aes_cipher_text_path, aes_key_path, aes_iv_path, output_path):
    # Step 1: Read AES cipher text (hex in text format) and convert to binary
    aes_cipher_text_bin = read_hex_file_as_bin(aes_cipher_text_path)

    # Step 2: Read AES key (binary format)
    aes_key_bin = read_file_as_bin(aes_key_path)

    # Step 3: Read AES IV (hex in text format) and convert to binary
    aes_iv_bin = read_hex_file_as_bin(aes_iv_path)

    # Step 4: Concatenate binary data (cipher text, key, IV)
    concatenated_bin_data = aes_cipher_text_bin + aes_key_bin + aes_iv_bin

    # Step 5: Convert the concatenated binary data to hexadecimal
    concatenated_hex_data = concatenated_bin_data.hex()

    # Step 6: Save the concatenated hex data to the output text file
    save_hex_to_file(concatenated_hex_data, output_path)
    print(f"Concatenated data saved in hexadecimal format at: {output_path}")

# Example usage
aes_cipher_text_path = "/content/sample_data/aes128_ciphertext_file.txt"  # Cipher text in hex (text format)
aes_key_path = "/content/sample_data/aes_key.bin"  # AES key in binary
aes_iv_path = "/content/sample_data/aes128_iv_file.txt"  # IV in hex (text format)
output_path = "/content/sample_data/aes_concate.txt"

concatenate_and_save_in_hex(aes_cipher_text_path, aes_key_path, aes_iv_path, output_path)


Concatenated data saved in hexadecimal format at: /content/sample_data/aes_concate.txt


In [ ]:
from cryptography.hazmat.primitives.asymmetric import rsa
from cryptography.hazmat.primitives import serialization
from cryptography.hazmat.backends import default_backend

# Generate RSA private key (2048 bits for better security)
private_key = rsa.generate_private_key(
    public_exponent=65537,
    key_size=2048,
    backend=default_backend()
)

# Serialize and save private key to a file
with open("private_key.pem", "wb") as private_file:
    private_file.write(
        private_key.private_bytes(
            encoding=serialization.Encoding.PEM,
            format=serialization.PrivateFormat.PKCS8,
            encryption_algorithm=serialization.NoEncryption()  # You can add encryption here if needed
        )
    )

# Generate the public key from the private key
public_key = private_key.public_key()

# Serialize and save public key to a file
with open("public_key.pem", "wb") as public_file:
    public_file.write(
        public_key.public_bytes(
            encoding=serialization.Encoding.PEM,
            format=serialization.PublicFormat.SubjectPublicKeyInfo
        )
    )

print("Keys generated and saved as 'private_key.pem' and 'public_key.pem'")


Keys generated and saved as 'private_key.pem' and 'public_key.pem'


In [ ]:
from Crypto.Cipher import PKCS1_OAEP
from Crypto.PublicKey import RSA
import binascii

def read_hex_file_as_bin(file_path):
    """Reads a file containing hex data and returns its content in binary format."""
    with open(file_path, 'r') as file:
        hex_data = file.read().strip()  # Strip any extra whitespace or newlines
        return bytes.fromhex(hex_data)  # Convert hex to binary

def save_bin_to_file(bin_data, output_path):
    """Saves binary data to a file."""
    with open(output_path, 'wb') as file:
        file.write(bin_data)

def rsa_encrypt(data_bin, public_key_path):
    """Encrypts binary data using RSA public key."""
    # Load the public key from file
    with open(public_key_path, 'rb') as f:
        public_key = RSA.import_key(f.read())

    # Create a cipher object using the public key
    cipher_rsa = PKCS1_OAEP.new(public_key)

    # Encrypt the data
    encrypted_data = cipher_rsa.encrypt(data_bin)
    return encrypted_data

def encrypt_and_save(aes_data_path, rsa_public_key_path, output_path):
    # Step 1: Read the concatenated hex data from file and convert to binary
    concatenated_data_bin = read_hex_file_as_bin(aes_data_path)

    # Step 2: Perform RSA encryption
    encrypted_data_bin = rsa_encrypt(concatenated_data_bin, rsa_public_key_path)

    # Step 3: Save the encrypted data to a binary file
    save_bin_to_file(encrypted_data_bin, output_path)
    print(f"RSA Encrypted data saved in binary format at: {output_path}")

# Example usage
aes_data_path = "/content/sample_data/aes_concate.txt"  # Concatenated AES cipher text, key, and IV in hex format
rsa_public_key_path = "/content/public_key.pem"  # RSA public key in PEM format
output_path = "/content/sample_data/rsa.bin"  # Output binary file path

encrypt_and_save(aes_data_path, rsa_public_key_path, output_path)


RSA Encrypted data saved in binary format at: /content/sample_data/rsa.bin


In [ ]:
def read_bin_file(file_path):
    """Reads a binary file and returns its content."""
    with open(file_path, 'rb') as file:
        return file.read()

def read_txt_file(file_path):
    """Reads a text file (for the double hash) and returns its content."""
    with open(file_path, 'r') as file:
        return file.read().strip()

def hex_to_bin(hex_str):
    """Converts a hex string to its binary representation."""
    return ''.join(format(int(char, 16), '04b') for char in hex_str)

def save_bin_file(output_path, bin_data):
    """Saves binary data to a binary file."""
    with open(output_path, 'wb') as file:
        file.write(bin_data)

def concatenate_bin(rsa_bin_path, double_hash_path, output_path):
    """Concatenates the RSA binary and the double-hashed value, saves as binary."""
    # Read the RSA binary file
    rsa_bin = read_bin_file(rsa_bin_path)

    # Read the double hash (assumed to be in hex format) from the text file
    double_hash_hex = read_txt_file(double_hash_path)

    # Convert the double hash from hex to binary format
    double_hash_bin = hex_to_bin(double_hash_hex)

    # Convert the binary string (0's and 1's) to bytes
    double_hash_bytes = int(double_hash_bin, 2).to_bytes((len(double_hash_bin) + 7) // 8, byteorder='big')

    # Concatenate RSA binary and double hash bytes
    concatenated_data = rsa_bin + double_hash_bytes

    # Save the concatenated data to the output file in binary format
    save_bin_file(output_path, concatenated_data)

    print(f"Concatenated binary data saved at: {output_path}")

# Example usage
rsa_bin_path = "/content/sample_data/rsa.bin"  # Path to the RSA binary file
double_hash_path = "/content/sample_data/double_hash.txt"  # Path to the double hash text file (hex format)
output_path = "/content/sample_data/output_encoded.bin"    # Path to save the concatenated binary

concatenate_bin(rsa_bin_path, double_hash_path, output_path)


Concatenated binary data saved at: /content/sample_data/output_encoded.bin


In [ ]:
from PIL import Image
import numpy as np

def read_bin_file(file_path):
    """Reads a binary file and returns its content."""
    with open(file_path, 'rb') as file:
        return file.read()

def bin_to_binary_string(bin_data):
    """Converts binary data to a binary string."""
    return ''.join(format(byte, '08b') for byte in bin_data)

def embed_into_image(binary_data, image_path, output_image_path):
    """Embeds binary data into an image using LSB technique."""
    # Load the image
    img = Image.open(image_path)
    img_array = np.array(img)

    # Ensure the image can fit the binary data
    if len(binary_data) > img_array.size:
        raise ValueError("Image is too small to hold the data.")

    # Convert the image to RGB
    img_array = img_array.astype(np.uint8)

    # Embed data into the image
    data_index = 0
    for i in range(img_array.shape[0]):
        for j in range(img_array.shape[1]):
            for k in range(img_array.shape[2]):  # For each RGB channel
                if data_index < len(binary_data):
                    # Modify the least significant bit of each color channel
                    img_array[i, j, k] = (img_array[i, j, k] & 0xFE) | int(binary_data[data_index])
                    data_index += 1

    # Save the stego image
    stego_img = Image.fromarray(img_array)
    stego_img.save(output_image_path)
    print(f"Stego image saved at: {output_image_path}")

# Example usage
rsa_bin_file_path = "/content/sample_data/output_encoded.bin"  # RSA encrypted binary data
image_path = "/content/sample_data/bird-8788491_1280.webp"  # Input image file
output_image_path = "/content/sample_data/output_stego_image.png"  # Output stego image file

# Read the RSA binary data
rsa_bin_data = read_bin_file(rsa_bin_file_path)

# Convert the binary data to a binary string
binary_data = bin_to_binary_string(rsa_bin_data)

# Embed the binary data into the image
embed_into_image(binary_data, image_path, output_image_path)


Stego image saved at: /content/sample_data/output_stego_image.png


In [ ]:
def read_bin_file(file_path):
    """Reads a binary file and returns its content."""
    with open(file_path, 'rb') as file:
        return file.read()

def bin_to_binary_string(bin_data):
    """Converts binary data to a binary string."""
    return ''.join(format(byte, '08b') for byte in bin_data)

def print_binary_file(file_path):
    """Reads the binary file, prints the binary string and the number of bits."""
    binary_data = read_bin_file(file_path)
    binary_string = bin_to_binary_string(binary_data)

    # Calculate the number of bits
    num_bits = len(binary_string)

    # Print the binary content and the number of bits
    print(f"Binary content of {file_path}:")
    print(binary_string)
    print(f"\nTotal number of bits: {num_bits}")

# Example usage
binary_file_path = "/content/sample_data/decrypted_aes_concate.bin"  # Replace with the actual path to your binary file
print_binary_file(binary_file_path)


Binary content of /content/sample_data/decrypted_aes_concate.bin:
11001011011000100110110100111100001111000111001011011000010111011111000101011100010001000011101000101010110010000110111100010111111010010010100011011111000111111100011111001000001101001010111011000010010000101111000001100011001110010111100010001111011110001100111101110011110010111110000111101111010110111000110100100100110110110110101010110111111111100110000101101110101000001000111000101011110001101010110001111000110101010001010111110100111011000001110101011110111000011001001001011001000010101000000111010011

Total number of bits: 512


In [ ]:
def read_bin_file(file_path):
    """Reads a binary file and returns its content."""
    with open(file_path, 'rb') as file:
        return file.read()

def bin_to_hex_string(bin_data):
    """Converts binary data to a hexadecimal string."""
    return bin_data.hex()

def print_hex_file(file_path):
    """Reads the binary file and prints the hexadecimal string and the number of bytes."""
    binary_data = read_bin_file(file_path)
    hex_string = bin_to_hex_string(binary_data)

    # Calculate the number of bytes
    num_bytes = len(binary_data)

    # Print the hexadecimal content and the number of bytes
    print(f"Hexadecimal content of {file_path}:")
    print(hex_string)
    print(f"\nTotal number of bytes: {num_bytes}")

# Example usage
binary_file_path = "/content/sample_data/rsa.bin"  # Replace with the actual path to your binary file
print_hex_file(binary_file_path)


Hexadecimal content of /content/sample_data/rsa.bin:
9136e0bc0ef02fc2c039b390b7412265dbb346dce818c36d2e70605539ec54289afb9acd5fd7bc3cf59ae0896e0fc5cfac24ae14d97845a0545f83bbaecaf832257ccdaa1a984cf9d3e69d6c5a2b2adfcd800952ceda9e86c71dd6181b5979bc1d3c533af8f68618f978dbf6f5370771448cb7fd3cb3f43415e278bef00a30677b3bb7daccc0c92c2e40ff877ac1ada5b19beeeb6688d535423ed6ce7b1610c1f7e6b274a3f5781855463aebb9719f90dbe19e60a6851820488ec3c1caa74ccef3b675616b0aa7ab9d30cd79633ca5d2cd21f19ff0468db0bcea91d3a0e7d00fbfbcb8e44f69ec8f2e0493f7d57b04e2be47a1a0a1be9cd924f86bdb54b05220

Total number of bytes: 256


In [ ]:
from PIL import Image

def extract_lsb_data(stego_image_path):
    """Extracts hidden binary data from an image using the Least Significant Bit (LSB) technique."""
    img = Image.open(stego_image_path)
    img_data = img.convert("RGB")
    pixels = list(img_data.getdata())

    # Extract LSB from each color channel and form the binary string
    binary_data = ""
    for pixel in pixels:
        for color in pixel:  # R, G, B channels
            binary_data += bin(color)[-1]  # Get the least significant bit

    return binary_data

def save_bin_file(output_path, binary_data, num_bytes):
    """Saves a specified number of bytes of binary data to a binary file."""
    # Convert the binary string to bytes
    byte_data = int(binary_data[:num_bytes * 8], 2).to_bytes(num_bytes, byteorder='big')

    with open(output_path, 'wb') as file:
        file.write(byte_data)

def decode_stego_image(stego_image_path, output_bin_path, num_bytes=288):
    """Decodes the hidden binary data from the stego image and saves the first specified number of bytes as a binary file."""
    # Extract the binary data from the image using LSB method
    binary_data = extract_lsb_data(stego_image_path)

    print(f"Extracted binary data (first 100 bits): {binary_data[:100]}...")  # Print part of the binary data

    # Save the first 288 bytes of binary data to a file
    save_bin_file(output_bin_path, binary_data, num_bytes)
    print(f"First {num_bytes} bytes of binary data saved to: {output_bin_path}")

# Example usage
stego_image_path = "/content/sample_data/output_stego_image.png"  # Path to the stego image
output_bin_path = "/content/sample_data/output_decoded.bin"  # Path to save the extracted binary

decode_stego_image(stego_image_path, output_bin_path)


Extracted binary data (first 100 bits): 1001000100110110111000001011110000001110111100000010111111000010110000000011100110110011100100001011...
First 288 bytes of binary data saved to: /content/sample_data/output_decoded.bin


In [ ]:
def separate_rsa_and_hash(decoded_bin_path, rsa_output_path, hash_output_path):
    """Separates RSA-encrypted binary and double hash binary from the decoded binary file."""

    # Step 1: Load the decoded binary data from file
    with open(decoded_bin_path, 'rb') as f:
        decoded_data = f.read()

    # Step 2: Define sizes (in bytes)
    rsa_size = 256  # RSA-2048 encrypted AES key (256 bytes)
    hash_size = 48   # MD5 + SHA-256 (32 bytes for SHA-256 and 16 bytes for MD5)

    # Step 3: Split the binary data
    rsa_binary = decoded_data[:rsa_size]  # First 256 bytes are RSA-encrypted data
    double_hash_binary = decoded_data[rsa_size:rsa_size + hash_size]  # Next 48 bytes are the double hash

    # Step 4: Convert the double hash binary into hex format
    double_hash_hex = double_hash_binary.hex()

    # Step 5: Save the RSA binary and double hash separately
    with open(rsa_output_path, 'wb') as rsa_file:
        rsa_file.write(rsa_binary)
    print(f"RSA-encrypted binary saved to: {rsa_output_path}")

    with open(hash_output_path, 'w') as hash_file:
        hash_file.write(double_hash_hex)
    print(f"Double hash (in hex format) saved to: {hash_output_path}")

    # Debug prints
    print(f"RSA-encrypted binary (first 64 bytes): {rsa_binary[:64].hex()}...")
    print(f"Double hash (hex): {double_hash_hex}")

# Example usage:
decoded_bin_path = "/content/sample_data/output_decoded.bin"  # Path to the decoded binary file
rsa_output_path = "/content/sample_data/output_rsa_binary.bin"     # Path to save the RSA-encrypted binary
hash_output_path = "/content/sample_data/output_double_hash_hex.txt"      # Path to save the double hash in hex format

# Call the function to separate and save RSA binary and double hash
separate_rsa_and_hash(decoded_bin_path, rsa_output_path, hash_output_path)


RSA-encrypted binary saved to: /content/sample_data/output_rsa_binary.bin
Double hash (in hex format) saved to: /content/sample_data/output_double_hash_hex.txt
RSA-encrypted binary (first 64 bytes): 9136e0bc0ef02fc2c039b390b7412265dbb346dce818c36d2e70605539ec54289afb9acd5fd7bc3cf59ae0896e0fc5cfac24ae14d97845a0545f83bbaecaf832...
Double hash (hex): 290928677845ee57a805b11badfb292419a11a4d151e47ae6d9c1d6011164666


In [ ]:
from Crypto.Cipher import PKCS1_OAEP
from Crypto.PublicKey import RSA

def rsa_decrypt(rsa_binary_path, private_key_path, output_path):
    """Decrypts RSA-encrypted binary data using the provided RSA private key and saves the output to a file."""
    # Load the RSA private key
    with open(private_key_path, 'rb') as key_file:
        private_key = RSA.import_key(key_file.read())

    # Create RSA decryption cipher
    cipher_rsa = PKCS1_OAEP.new(private_key)

    # Read the RSA-encrypted binary data
    with open(rsa_binary_path, 'rb') as enc_file:
        encrypted_data = enc_file.read()

    # Decrypt the data
    decrypted_data = cipher_rsa.decrypt(encrypted_data)

    # Save the decrypted data to a file
    with open(output_path, 'wb') as output_file:
        output_file.write(decrypted_data)

    print(f"Decrypted data saved to: {output_path}")

# Example usage
rsa_binary_path = "/content/sample_data/output_rsa_binary.bin"  # Path to RSA encrypted binary data
private_key_path = "/content/private_key.pem"   # Path to the RSA private key
output_path = "/content/sample_data/decrypted_aes_concate.bin"    # Path to save the decrypted output

rsa_decrypt(rsa_binary_path, private_key_path, output_path)


Decrypted data saved to: /content/sample_data/decrypted_aes_concate.bin


In [ ]:
from Crypto.PublicKey import RSA
from Crypto.Cipher import PKCS1_OAEP
import binascii

def rsa_decrypt(rsa_bin_path, private_key_path, output_aes_concat_hex_path):
    """Decrypt RSA encrypted data and save the AES concatenated output in hex format."""

    # Step 1: Load the RSA-encrypted binary data
    with open(rsa_bin_path, 'rb') as file:
        rsa_encrypted_data = file.read()

    print(f"Loaded RSA encrypted data (first 100 bytes): {binascii.hexlify(rsa_encrypted_data[:100])}...")

    # Step 2: Load the private key
    with open(private_key_path, 'rb') as file:
        private_key = RSA.import_key(file.read())

    # Step 3: Decrypt the data using RSA
    cipher_rsa = PKCS1_OAEP.new(private_key)
    decrypted_data = cipher_rsa.decrypt(rsa_encrypted_data)

    print(f"Decrypted AES concatenated data (first 100 bytes): {binascii.hexlify(decrypted_data[:100])}...")

    # Step 4: Convert decrypted data to hex format and save to file
    hex_data = binascii.hexlify(decrypted_data).decode('utf-8')
    with open(output_aes_concat_hex_path, 'w') as output_file:
        output_file.write(hex_data)

    print(f"AES concatenated data saved in hex format at: {output_aes_concat_hex_path}")

# Example usage
rsa_bin_path = "/content/sample_data/output_rsa_binary.bin"  # Path to the RSA encrypted binary file
private_key_path = "/content/private_key.pem"  # Path to the RSA private key file
output_aes_concat_hex_path = "/content/sample_data/decrypted_aes_concate.txt"  # Path to save the AES concatenated data in hex format

rsa_decrypt(rsa_bin_path, private_key_path, output_aes_concat_hex_path)


Loaded RSA encrypted data (first 100 bytes): b'9136e0bc0ef02fc2c039b390b7412265dbb346dce818c36d2e70605539ec54289afb9acd5fd7bc3cf59ae0896e0fc5cfac24ae14d97845a0545f83bbaecaf832257ccdaa1a984cf9d3e69d6c5a2b2adfcd800952ceda9e86c71dd6181b5979bc1d3c533a'...
Decrypted AES concatenated data (first 100 bytes): b'cb626d3c3c72d85df15c443a2ac86f17e928df1fc7c834aec242f06339788f78cf73cbe1ef5b8d24db6ab7fe616ea08e2bc6ac78d515f4ec1d5ee192590a81d3'...
AES concatenated data saved in hex format at: /content/sample_data/decrypted_aes_concate.txt


In [ ]:
def separate_aes_concat_v2(concat_txt_path, aes_cipher_text_path, aes_key_bin_path, aes_iv_txt_path):
    """Separates AES concatenated text into AES cipher text, AES key (bin format), and AES initialization vector (IV)."""

    # Step 1: Read the concatenated AES data from the text file
    with open(concat_txt_path, 'r') as file:
        concat_hex_data = file.read().strip()  # Hex format concatenated data

    # Convert hex string to bytes
    concat_data = bytes.fromhex(concat_hex_data)

    # Step 2: Extract the AES IV, Key, and Cipher Text
    aes_iv_size = 16   # AES IV size (16 bytes)
    aes_key_size = 16  # AES-128 key size (16 bytes)

    aes_iv = concat_data[-aes_iv_size:]  # Last 16 bytes
    aes_key = concat_data[-(aes_iv_size + aes_key_size):-aes_iv_size]  # Next 16 bytes before the IV
    aes_cipher_text = concat_data[:-(aes_iv_size + aes_key_size)]  # Remaining bytes are the cipher text

    print(f"AES IV (hex): {aes_iv.hex()}")
    print(f"AES Key (hex): {aes_key.hex()}")
    print(f"AES Cipher Text (first 100 bytes in hex): {aes_cipher_text[:100].hex()}...")

    # Step 3: Save each part to its respective file

    # Save AES cipher text in hex format
    with open(aes_cipher_text_path, 'w') as file:
        file.write(aes_cipher_text.hex())
    print(f"AES cipher text saved to: {aes_cipher_text_path}")

    # Save AES key in binary format
    with open(aes_key_bin_path, 'wb') as file:
        file.write(aes_key)
    print(f"AES key (bin) saved to: {aes_key_bin_path}")

    # Save AES IV in hex format
    with open(aes_iv_txt_path, 'w') as file:
        file.write(aes_iv.hex())
    print(f"AES IV saved to: {aes_iv_txt_path}")

# Example usage
concat_txt_path = "/content/sample_data/decrypted_aes_concate.txt"  # Path to the concatenated AES text file (in hex format)
aes_cipher_text_path = "/content/sample_data/decrypted_aes_cipher.txt"  # Path to save the AES cipher text (hex format)
aes_key_bin_path = "/content/sample_data/decrypted_aes_key.bin"  # Path to save the AES key (binary format)
aes_iv_txt_path = "/content/sample_data/decrypted_aes_iv.txt"  # Path to save the AES initialization vector (hex format)

separate_aes_concat_v2(concat_txt_path, aes_cipher_text_path, aes_key_bin_path, aes_iv_txt_path)


AES IV (hex): 2bc6ac78d515f4ec1d5ee192590a81d3
AES Key (hex): cf73cbe1ef5b8d24db6ab7fe616ea08e
AES Cipher Text (first 100 bytes in hex): cb626d3c3c72d85df15c443a2ac86f17e928df1fc7c834aec242f06339788f78...
AES cipher text saved to: /content/sample_data/decrypted_aes_cipher.txt
AES key (bin) saved to: /content/sample_data/decrypted_aes_key.bin
AES IV saved to: /content/sample_data/decrypted_aes_iv.txt


In [ ]:
from Crypto.Cipher import AES
from Crypto.Util.Padding import unpad

def aes_decrypt(cipher_text_path, aes_key_bin_path, aes_iv_txt_path, output_plain_text_path):
    """Performs AES decryption to retrieve the plain text and save it to a file."""

    # Step 1: Read the AES cipher text from the file (in hex format)
    with open(cipher_text_path, 'r') as file:
        aes_cipher_text_hex = file.read().strip()

    aes_cipher_text = bytes.fromhex(aes_cipher_text_hex)  # Convert hex to bytes

    # Step 2: Read the AES key from the binary file
    with open(aes_key_bin_path, 'rb') as file:
        aes_key = file.read()  # Binary key

    # Step 3: Read the AES IV from the text file (in hex format)
    with open(aes_iv_txt_path, 'r') as file:
        aes_iv_hex = file.read().strip()

    aes_iv = bytes.fromhex(aes_iv_hex)  # Convert hex to bytes

    # Step 4: Perform AES decryption (AES-128 CBC mode)
    cipher_aes = AES.new(aes_key, AES.MODE_CBC, aes_iv)
    plain_text = unpad(cipher_aes.decrypt(aes_cipher_text), AES.block_size)

    # Step 5: Save the decrypted plain text to the output file
    with open(output_plain_text_path, 'w') as file:
        file.write(plain_text.decode('utf-8'))

    print(f"Decrypted plain text saved to: {output_plain_text_path}")
    print(f"Decrypted Plain Text: {plain_text.decode('utf-8')}")

# Example usage
cipher_text_path = "/content/sample_data/decrypted_aes_cipher.txt"  # Path to the AES cipher text file (hex format)
aes_key_bin_path = "/content/sample_data/decrypted_aes_key.bin"  # Path to the AES key (binary format)
aes_iv_txt_path = "/content/sample_data/decrypted_aes_iv.txt"  # Path to the AES IV file (hex format)
output_plain_text_path = "/content/sample_data/decrypted_plain_text.txt"  # Path to save the decrypted plain text

aes_decrypt(cipher_text_path, aes_key_bin_path, aes_iv_txt_path, output_plain_text_path)


Decrypted plain text saved to: /content/sample_data/decrypted_plain_text.txt
Decrypted Plain Text: hi, how are you?


In [ ]:
import hashlib

def calculate_double_hash(plain_text):
    """Performs double hashing (MD5 followed by SHA-256) on the plain text."""
    # Step 1: Perform MD5 hash
    md5_hash = hashlib.md5(plain_text.encode()).hexdigest()

    # Step 2: Perform SHA-256 hash on the MD5 hash
    sha256_hash = hashlib.sha256(md5_hash.encode()).hexdigest()

    return sha256_hash

def compare_hashes(decrypted_plain_text_path, decoded_hash_value_path):
    """Compares the double hash of the decrypted plain text with the decoded hash value."""

    # Step 3: Read the decrypted plain text
    with open(decrypted_plain_text_path, 'r') as file:
        decrypted_plain_text = file.read().strip()

    print(f"Decrypted Plain Text: {decrypted_plain_text}")

    # Step 4: Perform double hashing (MD5 followed by SHA-256)
    calculated_hash = calculate_double_hash(decrypted_plain_text)
    print(f"Calculated Double Hash: {calculated_hash}")

    # Step 5: Read the decoded hash value from the file (in hex format)
    with open(decoded_hash_value_path, 'r') as file:
        decoded_hash_value = file.read().strip()

    print(f"Decoded Hash Value: {decoded_hash_value}")

    # Step 6: Compare the calculated hash with the decoded hash value
    if calculated_hash == decoded_hash_value:
        print("Successfully performed image steganography!")
    else:
        print("Message is corrupted.")

# Example usage
decrypted_plain_text_path = "/content/sample_data/decrypted_plain_text.txt"  # Path to the decrypted plain text file
decoded_hash_value_path = "/content/sample_data/output_double_hash_hex.txt"  # Path to the decoded hash value file (hex format)

compare_hashes(decrypted_plain_text_path, decoded_hash_value_path)



Decrypted Plain Text: hi, how are you?
Calculated Double Hash: 290928677845ee57a805b11badfb292419a11a4d151e47ae6d9c1d6011164666
Decoded Hash Value: 290928677845ee57a805b11badfb292419a11a4d151e47ae6d9c1d6011164666
Successfully performed image steganography!
